In [28]:
import GetOldTweets3 as got
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pickle
import plotly.graph_objects as go


In [2]:
%%time

text_query = 'USA Election 2020'
count = 100
# Creation of query object
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setMaxTweets(count)
# Creation of list that contains all tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

CPU times: user 430 ms, sys: 39.9 ms, total: 469 ms
Wall time: 11 s


In [3]:
# Creating list of chosen tweet data
text_tweets = [[tweet.date, tweet.text] for tweet in tweets]

In [5]:
text_tweets[0]

datetime.datetime(2020, 6, 6, 16, 7, 34, tzinfo=datetime.timezone.utc)

In [8]:
df = pd.DataFrame(text_tweets, columns =['Date', 'Tweets']) 

In [37]:
def getTweets(query,count):
    text_query = query
    # Creation of query object
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setMaxTweets(count)
    # Creation of list that contains all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    text_tweets = [[tweet.date, tweet.text] for tweet in tweets]
    df = pd.DataFrame(text_tweets, columns =['Date', 'Tweets']) 
    return df


In [22]:
def preprocess(tweet):
    #LowerCase 
    tweet=tweet.lower()

    #Replacing URL
    tweet=tweet.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "URL")

    #Removing Username
    tweet=tweet.replace(r'@[^\s]+',"")

    #Removing Non-Alpha Numeric Chars
    tweet=tweet.replace(r'[^A-Za-z0-9 ]+',"")


    stop_words = stopwords.words('english') 
    text_tokens = word_tokenize(tweet)
    tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    

    #Lementize
    wordlem = WordNetLemmatizer()
    tokens_without_sw=[wordlem.lemmatize(word) for word in tokens_without_sw]
    filtered_sentence = (" ").join(tokens_without_sw)



    return filtered_sentence


def load_models():
    # Load the vectoriser.
    file = open('../Models/tfidf-ngram-(1,3).pickle', 'rb')
    vectorizer = pickle.load(file)
    file.close()
    
    # Load the LR Model.
    file = open('../Models/svc.pickle', 'rb')
    lr = pickle.load(file)
    file.close()
    
    return vectorizer, lr



def predict(vectorizer,model,tweets):

    print ("----------------PreProcessing--------------------------")
    preproc=[]
    for tweet in tweets:
        preproc.append(preprocess(tweet))

    print ("----------------Vectorising--------------------------")
    vect=vectorizer.transform(preproc)
    
    print ("----------------Predicting--------------------------")
    sent=model.predict(vect)


    data = []
    for text, pred in zip(tweets, sent):
        data.append((text,pred))

    df=pd.DataFrame(data,columns=["Tweets","Sentiment"])
    df = df.replace([0,1], ["Negative","Positive"])

    return df



In [38]:
%%time
tw1=getTweets('USA Election 2020',100)
tw2=getTweets('USA Election 2016',100)

CPU times: user 931 ms, sys: 42.3 ms, total: 973 ms
Wall time: 21.7 s


In [39]:
vect,model=load_models()
tw1_pred=predict(vect,model,tw1["Tweets"].tolist())
tw2_pred=predict(vect,model,tw2["Tweets"].tolist())

----------------PreProcessing--------------------------
----------------Vectorising--------------------------
----------------Predicting--------------------------
----------------PreProcessing--------------------------
----------------Vectorising--------------------------
----------------Predicting--------------------------


In [40]:
tw1_pred["Date"]=tw1["Date"]
tw2_pred["Date"]=tw2["Date"]
# tdf["Date"]=df["Date"]

In [41]:
tw1_pred

,Tweets,Sentiment,Date
0,#USA should look and cast vote for #BetterCand...,Positive,2020-06-06 17:46:38+00:00
1,Equity Markets 2020 Things to note for equity ...,Negative,2020-06-06 17:38:49+00:00
2,Nazis from Russia. Those are Putin forces that...,Negative,2020-06-06 17:38:24+00:00
3,He's desperate to get everyone in USA to get s...,Negative,2020-06-06 17:35:43+00:00
4,No matter who wins this election (if there eve...,Negative,2020-06-06 17:31:30+00:00
...,...,...,...
95,#NEW #Texas Presidential Election Poll Donald ...,Positive,2020-06-05 15:35:54+00:00
96,TRUMP WILL NOT LEAVE EVEN THOUGH HE WILL LOSE ...,Negative,2020-06-05 15:21:15+00:00
97,This man is using troll farms (weapons of war)...,Negative,2020-06-05 15:16:06+00:00
98,I am in no doubt coronavirus was an act of sab...,Positive,2020-06-05 15:06:27+00:00


In [44]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=tw1_pred["Date"],y=tw1_pred["Sentiment"],name='USA Election 2020'))
fig.show()

In [53]:
fig=go.Figure()
fig.add_trace(go.Pie(labels=['Negative','Positive'],values=val,name='Election 2020'))
fig.show()

In [60]:
fig=go.Figure()
fig.add_trace(go.Bar(x=['Negative','Positive'],y=val,name='Election 2020'))
fig.add_trace(go.Bar(x=['Negative','Positive'],y=val2,name='Election 2016'))

fig.update_layout(title='Election 2016 v Election 2020',title_x=0.5,
                   xaxis_title='Sentiment',
                   yaxis_title='Number of Tweets')

fig.show()

In [50]:
val=tw1_pred["Sentiment"].value_counts().values
val2=tw2_pred["Sentiment"].value_counts().values